problem3：在实际疫情情况下的多轮检测
* 基本假设：
  * 1、感染人数符合二项分布N（n,p），单独检测时间为10min/人，混合检测为40min/组
  * 2、检测试剂的置信度为100%：由于检测一般经过两次及以上过程，而两次检测均出错的几率可以基本忽略
  * 3、二分或四分检测按原编号顺序组内依次执行
* 数据样本：同上problem1、2，采用武汉疫情的基本数据
* 参数规定：
  * 总人数：n
  * 基础感染率：p
  * 单独检测总时间：t0 = 10min
  * 分组检测总时间：t1 = 40min
  * 检测试剂的准确率情况：假阴性率β，真阳性TP，假阳性FP，假阴性FN，真阴性TN，或由灵敏度Sn = 1 - β得到，实际上由官方数据中（当日新增确诊）/（当日新增确诊 + 当日新增疑似病例得到），即
* 核心问题：
  * 1、根据上一轮得到的结果（阳性组数量），对下一轮检测方法进行动态调整
  * 2、采取组合检测模型以最高的效率实现对本轮阳性样本的溯源
  * 3、对真实情况进行考虑，增加假阴性、潜伏期等影响因素
* 基本思路：
  * 1、用稀疏矩阵代表感染者的分布情况：0为未感染，1为已感染
  * 2、采用惩罚（奖励）系数对采样方案进行更新迭代，选取的方案为二分检测或四分检测等（不知道咋弄）
  * 3、使用蒙特卡洛算法对时间序列进行预测
  * 4、预测流程思路：
    * 1）起始阶段：通过初期的检测近似统计确定患病率p
    * 2）中间主要阶段：按照患病率p确定最佳分组人数，分组检测；
    * 3）收尾阶段：此时样本数较小，可以逐例检测。此阶段对整体检测效率影响不大。
* 模型优化：
  * 纳入假阴性等影响因素，尽管在概率上对确诊人数的影响完全可以忽略，但一旦发生，则会对实际防疫过程将产生巨大的影响。

In [1]:
import random
import math
import matplotlib.pyplot as plt
from scipy.sparse import rand
import numpy as np

n = 250000
t0 = 10
t1 = 40
p = 0.069
# 感染总人数
N_infect = int(n * p)
# 单独检测总时间
T0 = n * t0

C = 100 # 循环次数
count = 0 # 循环计数器

# 枚举法计算二分需要的次数
def binary_devide(flag, temp):
    count = 0
    res = np.zeros(4)
    if (flag == 1):
        count += 2 * t1 + 2 * t0
    if (flag == 2):
        for j in range(len(temp)):
            if(temp[j] != 0):
                res[j] = 1
            if ((res[0] == 1 and res[1] == 1) or (res[2] == 1 and res[3] == 1)):
                count += 2 * t1 + 2 * t0
            else:
                count += 2 * t1 + 4 * t0
    else:
        count += 2 * t1 + 4 * t0
    return count

# 四分法
def quarter_devide():
    count = 0
    count += 4 * t0
    return count

def test_per_round_binary():
    time = 0
    # 生成稀疏矩阵
    M = rand((250000 // 4), 4, 0.069)
    M = M.toarray()
    for i in range(len(M)):
        time += t1
        # 记录阳性样本出现位置
        temp = []
        flag = 0
        for j in range(len(M[i])):
            if M[i][j] != 0:
                temp.append(j)
                flag += 1
            else:
                temp.append(0)
            if (flag != 0):
                time += binary_devide(flag,temp)
    return time

def test_per_round_quarter():
    time = 0
    # 生成稀疏矩阵
    M = rand((250000 // 4), 4, 0.069)
    M = M.toarray()
    for i in range(len(M)):
        time += t1
        # 记录阳性样本出现位置
        flag = False
        for j in range(len(M[i])):
            if M[i][j] != 0:
                flag = True
                break
        if (flag):
            time += quarter_devide()
    return time

#print(test_per_round_binary() / T0)
#print(test_per_round_quarter() / T0)

# 分别记录二分法和四分法的效率比（与单独检测作商）
time1 = 0
time2 = 0

while (count < C):
    time1 += test_per_round_binary()
    time2 += test_per_round_quarter()
    count += 1

print(time1 / C / T0)
print(time2 / C / T0)

1000
2000
1.971925588
0.498728664
